# Schnorr Signature
We now present a simple implementation for Schnoor Signatures. These are a way to digitally sign a message, proving that it has not been tampered with. It is an extended version of the discrete log proofs that we showed before. 
First of all, we assume that the participants agreed on a $\mathbb{GF}(N)$ field, where $N$ is a safe prime of the form $2p + 1$ where $p$ is prime. Also the parties must have agreed on a primitive root of $N$, say $g$. We assume that the prover has a private key $x$ and a public key $\beta = g^x$. Then, to sign a message $M$, the protocol is as follows:
1. The prover chooses a random $r$, and computes $\gamma = g^r$. It then computes $e \equiv H(\gamma || M)$ and $y \equiv r - ex$. The signature is not the pair $(e, y)$
2. The verifier (or anyone else) can verify the message by computing $\gamma' = g^y \beta^e$, and then verifying that $e = H(\gamma' || M)$  

As always, we start by selecting our common parameters

In [6]:
bits = 128
t = 10

def safe_prime(nbits):
    while True:
        p = random_prime(2^nbits-1, false, 2^(nbits-1))
        if ZZ((p+1)/2).is_prime():
            return p
        
import hashlib
from sage.crypto.util import ascii_to_bin

def H(*args):
    hs = hashlib.sha1()
    for a in args:
        hs.update(str(a))
    return hs.hexdigest()

def hash_to_int(hs):
    return ZZ('0x' + hs) % 2^t
        
N = safe_prime(bits)
F = FiniteField(N)
g = F.multiplicative_generator()

First, we set the private and public key of the prover. Again, $x$ is considered private

In [7]:
x = randint(1, N - 1)
beta = g^x
beta

(518, -79495475519976319670810223090308003215876)

Now, suppose we want to sign the message "Hello there". We follow the steps in the protocol and the resulting certificate is the pair $(e, y)$

In [ ]:
M = "Hello there"
r = randint(1, N - 1)
gamma = g^r
e = hash_to_int(H(gamma, M))
y = r - e * x
(e, y)

Now, the verifier can use the certificate to sign the message, and check that it has not been tampered

In [8]:
gamma_prime = (g^y) * (beta^e)
e == hash_to_int(H(gamma_prime, M))

True

Instead, if the message that we saw was say "Hello thre", then we can see that the verifier would no longer accept

In [9]:
e == hash_to_int(H(gamma_prime, "Hello thre"))

False